In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('data/S-DoT_NATURE_2023년(2023.01.01~2024.01.01)/S-DoT_NATURE_2023.12.25-12.31.csv', encoding = 'euc-kr')

C:\Users\User\AppData\Local\Temp\ipykernel_27364\3938023480.py:5: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/S-DoT_NATURE_2023년(2023.01.01~2024.01.01)/S-DoT_NATURE_2023.12.25-12.31.csv', encoding = 'euc-kr')


In [3]:
print(df.head())
# 분석에 필요한 컬럼만 선택
df_env = df[['측정시간', '자치구', '온도 평균(℃)', '습도 평균(%)']].copy()

# '온도 평균(℃)'와 '습도 평균(%)'을 숫자형으로 변환 (오류 무시)
# isdigit()이 아닌 pd.to_numeric을 사용하고 'coerce'를 통해 NaN으로 변환하여 오류 처리
df_env['온도 평균(℃)'] = pd.to_numeric(df_env['온도 평균(℃)'], errors='coerce')
df_env['습도 평균(%)'] = pd.to_numeric(df_env['습도 평균(%)'], errors='coerce')

# '측정시간'을 datetime 타입으로 변환하고, 날짜/시간 정보 추출
df_env['측정시간'] = pd.to_datetime(df_env['측정시간'], format='%Y-%m-%d_%H:%M:%S', errors='coerce')
df_env['날짜'] = df_env['측정시간'].dt.date # 일별 분석을 위해 날짜만 추출
df_env['시간대'] = df_env['측정시간'].dt.hour # 시간대별 분석을 위해 시간만 추출

# 결측값(NaN)이 있는 행 제거 (온도/습도 데이터가 없는 경우)
df_env.dropna(subset=['온도 평균(℃)', '습도 평균(%)'], inplace=True)

                  측정시간                지역           자치구             행정동  \
0  2023-12-25_12:07:00   roads_and_parks   Jungnang-gu  Myeonmok4-dong   
1  2023-12-25_12:07:00  residential_area   Jungnang-gu  Seongnae1-dong   
2  2023-12-25_12:07:00       main_street  Eunpyeong-gu    Jingwan-dong   
3  2023-12-25_12:07:00  residential_area   Gwangjin-gu   Junggok2-dong   
4  2023-12-25_12:07:00   roads_and_parks   Jungnang-gu    Sinnae1-dong   

  온도 최대(℃) 온도 평균(℃) 온도 최소(℃) 습도 최대(%) 습도 평균(%) 습도 최소(%)  ... 암모니아 최대(ppm)  \
0      0.2      0.0     -0.3     72.0     71.0     69.0  ...          NaN   
1     -2.4     -2.5     -2.6     95.0     95.0     94.0  ...          NaN   
2     -1.8     -2.2     -2.5     76.0     75.0     73.0  ...          NaN   
3     -0.3     -0.5     -0.6     69.0     68.0     67.0  ...          NaN   
4     -3.5     -3.7     -3.7    100.0    100.0    100.0  ...          NaN   

  암모니아 평균(ppm) 암모니아 최소(ppm) 황화수소 최대(ppm) 황화수소 평균(ppm) 황화수소 최소(ppm) 오존 최대(ppm)  \
0          

In [4]:
# 예시: 일별/자치구별 평균 환경값 집계
df_daily_avg = df_env.groupby(['날짜', '자치구']).agg(
    일별_온도_평균=('온도 평균(℃)', 'mean'),
    일별_습도_평균=('습도 평균(%)', 'mean')
).reset_index()

print("--- 일별/자치구별 환경 데이터 집계 (샘플) ---")
print(df_daily_avg.head())

--- 일별/자치구별 환경 데이터 집계 (샘플) ---
           날짜            자치구  일별_온도_평균   일별_습도_평균
0  2023-12-25      Dobong-gu -1.885022  79.466595
1  2023-12-25  Dongdaemun-gu  0.805948  75.000000
2  2023-12-25     Dongjak-gu  0.902345  76.110553
3  2023-12-25   Eunpyeong-gu -2.314875  76.525773
4  2023-12-25     Gangbuk-gu  0.051276  77.235969


In [5]:
import pandas as pd
import glob
import os

def preprocess_file(file_path: str) -> pd.DataFrame:
    """
    단일 CSV 파일을 로드하고, 환경 변수를 정제하여 일별/자치구별 평균을 집계하는 함수.
    
    Args:
        file_path: 처리할 CSV 파일의 전체 경로
    
    Returns:
        일별/자치구별 집계된 환경 변수 DataFrame
    """
    try:
        # 1. 파일 로드 (인코딩 에러 대비)
        df = pd.read_csv(file_path, encoding='utf-8', low_memory=False)
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='euc-kr', low_memory=False)

    # 2. 필수 컬럼 선택 및 타입 변환
    try:
        df_env = df[['측정시간', '자치구', '온도 평균(℃)', '습도 평균(%)']].copy()
    except KeyError as e:
        print(f"Error: 파일 {os.path.basename(file_path)}에 필수 컬럼이 없습니다. ({e})")
        return pd.DataFrame() # 빈 DataFrame 반환

    # 숫자형으로 강제 변환 (coerce는 변환 실패 시 NaN으로 처리)
    df_env['온도 평균(℃)'] = pd.to_numeric(df_env['온도 평균(℃)'], errors='coerce')
    df_env['습도 평균(%)'] = pd.to_numeric(df_env['습도 평균(%)'], errors='coerce')

    # Datetime 변환 및 날짜/시간 추출
    df_env['측정시간'] = pd.to_datetime(df_env['측정시간'], format='%Y-%m-%d_%H:%M:%S', errors='coerce')
    df_env['날짜'] = df_env['측정시간'].dt.date 

    # 3. 결측값 처리 및 집계
    df_env.dropna(subset=['온도 평균(℃)', '습도 평균(%)'], inplace=True)
    
    # 일별/자치구별 평균 환경값 집계
    df_daily_avg = df_env.groupby(['날짜', '자치구']).agg(
        일별_온도_평균=('온도 평균(℃)', 'mean'),
        일별_습도_평균=('습도 평균(%)', 'mean')
    ).reset_index()

    print(f"✅ File processed: {os.path.basename(file_path)} -> {len(df_daily_avg)} rows aggregated.")
    return df_daily_avg

In [7]:
# 🚨 1. 파일이 위치한 폴더 경로 지정
# 예시: 스크립트가 실행되는 곳의 하위 폴더 'data_files'를 가정합니다.
# 실제 환경에 맞게 경로를 수정하세요.
folder_path = './data/S-DoT_NATURE_2023년(2023.01.01~2024.01.01)' 

# 🚨 2. glob을 사용하여 경로 내의 모든 CSV 파일 목록 가져오기
# pattern = os.path.join(folder_path, '*.csv')
all_csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# 3. 처리된 DataFrame들을 담을 리스트 초기화
processed_data_list = []

# 4. 파일 목록 순환 및 전처리 함수 적용
if not all_csv_files:
    print(f"⚠️ 경고: 경로 ({folder_path})에 CSV 파일이 없습니다. 경로를 확인해주세요.")
else:
    for file_path in all_csv_files:
        df_aggregated = preprocess_file(file_path)
        if not df_aggregated.empty:
            processed_data_list.append(df_aggregated)

# 5. 모든 집계된 데이터를 하나의 최종 데이터프레임으로 통합
if processed_data_list:
    final_combined_df = pd.concat(processed_data_list, ignore_index=True)
    
    # 최종 결과 확인 (중복 날짜/자치구 조합 제거)
    final_combined_df.drop_duplicates(subset=['날짜', '자치구'], keep='first', inplace=True)
    
    print("\n--- ✅ 최종 통합된 데이터프레임 구조 ---")
    print(f"총 행 수: {len(final_combined_df)}")
    print(final_combined_df.head())
    print("-" * 50)
    
    # 6. 다음 단계 분석을 위해 파일 저장 (선택 사항)
    # final_combined_df.to_csv('./final_aggregated_environment_data.csv', index=False)
else:
    print("❌ 최종 통합할 데이터가 없습니다.")

✅ File processed: S-DoT_NATURE_2023.01.01.csv -> 26 rows aggregated.
✅ File processed: S-DoT_NATURE_2023.01.02-01.08.csv -> 189 rows aggregated.
✅ File processed: S-DoT_NATURE_2023.01.09-01.15.csv -> 189 rows aggregated.
✅ File processed: S-DoT_NATURE_2023.01.16-01.22.csv -> 189 rows aggregated.
✅ File processed: S-DoT_NATURE_2023.01.23-01.29.csv -> 189 rows aggregated.
✅ File processed: S-DoT_NATURE_2023.01.30-02.05.csv -> 162 rows aggregated.
✅ File processed: S-DoT_NATURE_2023.02.06-02.12.csv -> 189 rows aggregated.
✅ File processed: S-DoT_NATURE_2023.02.13-02.19.csv -> 189 rows aggregated.
✅ File processed: S-DoT_NATURE_2023.02.20-02.26.csv -> 162 rows aggregated.
✅ File processed: S-DoT_NATURE_2023.02.27-03.05.csv -> 108 rows aggregated.
✅ File processed: S-DoT_NATURE_2023.03.06-03.12.csv -> 135 rows aggregated.
✅ File processed: S-DoT_NATURE_2023.03.13-03.19.csv -> 108 rows aggregated.
✅ File processed: S-DoT_NATURE_2023.03.20-03.26.csv -> 189 rows aggregated.
✅ File processed: S

In [8]:
final_combined_df.to_csv('./filtered_data/sdot.csv', index=False)

In [10]:
import pandas as pd

# 서울시 25개 자치구 영문 -> 한글 매핑 정의
district_map = {
    'Gangnam-gu': '강남구', 'Gangdong-gu': '강동구', 'Gangbuk-gu': '강북구', 
    'Gangseo-gu': '강서구', 'Gwanak-gu': '관악구', 'Gwangjin-gu': '광진구', 
    'Guro-gu': '구로구', 'Geumcheon-gu': '금천구', 'Nowon-gu': '노원구', 
    'Dobong-gu': '도봉구', 'Dongdaemun-gu': '동대문구', 'Dongjak-gu': '동작구', 
    'Mapo-gu': '마포구', 'Seodaemun-gu': '서대문구', 'Seocho-gu': '서초구', 
    'Seongdong-gu': '성동구', 'Seongbuk-gu': '성북구', 'Songpa-gu': '송파구', 
    'Yangcheon-gu': '양천구', 'Yeongdeungpo-gu': '영등포구', 'Yongsan-gu': '용산구', 
    'Eunpyeong-gu': '은평구', 'Jongno-gu': '종로구', 'Jung-gu': '중구', 
    'Jungnang-gu': '중랑구'
}

file_name = "./filtered_data/sdot.csv"
df = pd.read_csv(file_name)

# 1. 'Seoul_Grand_Park' 제외
df_filtered = df[(df['자치구'] != 'Seoul_Grand_Park') & (df['자치구'] != 'mobile_type')].copy()

# 2. 영문 자치구 명을 한글로 변경
df_filtered['자치구'] = df_filtered['자치구'].replace(district_map)

# 처리된 파일 저장
output_file_name = "./filtered_data/sdot_processed_ko.csv"
df_filtered.to_csv(output_file_name, index=False)

In [14]:
import pandas as pd

# 데이터 로드
df = pd.read_csv("./filtered_data/sdot_processed_ko.csv")

# '날짜' 기준으로 그룹화하고, '일별_습도_평균'의 평균을 계산
seoul_avg_humidity = df.groupby('날짜')[['일별_습도_평균','일별_온도_평균']].mean().reset_index()

# 컬럼명 변경
seoul_avg_humidity.columns = ['날짜', '서울시_평균_습도(%)','서울시_평균_온도']

# 결과 파일 저장
seoul_avg_humidity.to_csv("./filtered_data/seoul_daily_avg_humidity.csv", index=False)